In [6]:
import tensorflow as tf
from tensorflow import lite
from tensorflow.keras import models

In [7]:
# Parameters
keras_model_filename = 'wake_word_stop_model.h5'
tflite_filename = 'wake_word_stop_lite.tflite'
c_model_name = 'wake_word_stop_h' 

In [17]:
# Convert model to TF Lite model
model = models.load_model(keras_model_filename)
converter = lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter.allow_custom_ops=True
tflite_model = converter.convert()
open(tflite_filename, 'wb').write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpwxdgs3v7/assets


INFO:tensorflow:Assets written to: /tmp/tmpwxdgs3v7/assets
2024-04-14 16:56:06.356217: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-04-14 16:56:06.356238: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-04-14 16:56:06.356406: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpwxdgs3v7
2024-04-14 16:56:06.357387: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-04-14 16:56:06.357398: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpwxdgs3v7
2024-04-14 16:56:06.359968: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-04-14 16:56:06.390300: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmpwxdgs3v7
2024-04-14 16:56:06.401258: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

23296

In [22]:
# Function: Convert some hex value into an array for C programming
def hex_to_c_array(hex_data, var_name):

  c_str = ''

  # Create header guard
  c_str += '#ifndef ' + var_name.upper() + '_H\n'
  c_str += '#define ' + var_name.upper() + '_H\n\n'

  # Add array length at top of file
  c_str += '\nunsigned int ' + var_name + '_len = ' + str(len(hex_data)) + ';\n'

  # Declare C variable
  c_str += 'unsigned char ' + var_name + '[] = {'
  hex_array = []
  for i, val in enumerate(hex_data) :

    # Construct string from hex
    hex_str = format(val, '#04x')

    # Add formatting so each line stays within 80 characters
    if (i + 1) < len(hex_data):
      hex_str += ','
    if (i + 1) % 12 == 0:
      hex_str += '\n '
    hex_array.append(hex_str)

  # Add closing brace
  c_str += '\n ' + format(' '.join(hex_array)) + '\n};\n\n'

  # Close out header guard
  c_str += '#endif //' + var_name.upper() + '_H'

  return c_str

In [25]:
# Write TFLite model to a C source (or header) file
with open(c_model_name + '.h', 'w') as file:
  file.write(hex_to_c_array(tflite_model, c_model_name))

In [29]:
# Function #2: converting to tfLite
import numpy as np
import os

def convert_tflite_to_header(tflite_path, output_header_path):

    with open(tflite_path, 'rb') as tflite_file:
        tflite_content = tflite_file.read()
    
    
    hex_lines = [', '.join([f'0x{byte:02x}' for byte in tflite_content[i:i+12]]) for i in range(0, len(tflite_content), 12)]
    
    
    hex_array = ',\n  '.join(hex_lines)
    
    
    with open(output_header_path, 'w') as header_file:
        
        header_file.write('const unsigned char model[] = {\n  ')
        header_file.write(f'{hex_array}\n')
        header_file.write('};\n\n')
        
if __name__ == "__main__":
    tflite_path = 'wake_word_stop_lite.tflite'
    output_header_path = '2_wake_word_stop_h.h'

    convert_tflite_to_header(tflite_path, output_header_path)